In [41]:
import pandas as pd
result_filename = "../results/assignments_v3.csv"
hits_filename = "../cache/hits_v3.txt"
out_filename = "../results/data_analysis_v3.csv"

file = open(result_filename, "r")
df = pd.read_csv(file)
df = df.reset_index()

In [42]:
# clean string, lower, stem
# import nltk
# from nltk.stem.porter import PorterStemmer
# stemmer = PorterStemmer()

def preprocess(raw_str):
    clean_str = ''.join([letter for letter in raw_str.lower().replace('+',' ') if letter not in punc])
    clean_str_without_stopwords = ' '.join(set([word for word in clean_str.split() if word not in stopwords.words()]))
    return nlp(clean_str_without_stopwords)

import spacy
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nlp = spacy.load('en_core_web_md')
df = df.rename(columns={"Answer": "RawAnswer"})
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
df["CleanAnswer"] = pd.Series(dtype='object')
for index, row in df.iterrows():
    df.at[index,"CleanAnswer"] = preprocess(row["RawAnswer"])


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/henrywu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
# add ground truth
reference = {}
with open(hits_filename, "r") as hits_f:
    with open("../data/dev.csv", "r") as ref_f:
        ref_df = pd.read_csv(ref_f)
        for i, hit_id in enumerate(hits_f.readlines()):
            reference[hit_id.strip()] = ref_df.iloc[i]["labels"].strip().replace(";"," ")

df["GroundTruth"] = pd.Series(dtype='object')
for index, row in df.iterrows():
    df.at[index,"GroundTruth"] = preprocess(reference[row["HITId"]])

In [44]:
# average word vector
import numpy as np
df["AnswerWord2VectorAverage"] = pd.Series(dtype='object')
df["GroundTruthWord2VectorAverage"] = pd.Series(dtype='object')
df["similarity"] = pd.Series(dtype='float')
for index, row in df.iterrows():
    df.at[index,"AnswerWord2VectorAverage"] = np.array([token.vector for token in row["CleanAnswer"]]).mean(axis=0)
    df.at[index,"GroundTruthWord2VectorAverage"] = np.array([token.vector for token in row["GroundTruth"]]).mean(axis=0)
    df.at[index,"similarity"] = np.sum(df.iloc[index]["AnswerWord2VectorAverage"]*df.iloc[index]["GroundTruthWord2VectorAverage"])/np.linalg.norm(df.iloc[index]["AnswerWord2VectorAverage"])/np.linalg.norm(df.iloc[index]["GroundTruthWord2VectorAverage"])
    # 300
df

/var/folders/gq/hy95453j74zcwp8jxdtmwwsc0000gn/T/ipykernel_98497/859175069.py:7: RuntimeWarning: Mean of empty slice.
  df.at[index,"AnswerWord2VectorAverage"] = np.array([token.vector for token in row["CleanAnswer"]]).mean(axis=0)
/Users/henrywu/opt/anaconda3/envs/mturk/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,index,AssignmentId,WorkerId,HITId,AutoApprovalTime,AcceptTime,SubmitTime,RawAnswer,IsHint,CleanAnswer,GroundTruth,AnswerWord2VectorAverage,GroundTruthWord2VectorAverage,similarity
0,0,304SM51WAHO8MS4LKEI0BCZ0AA7BSC,A2FEJ4U14OS2M6,3URJ6VVYV276HHTQ8GM3U9NDDUVO47,2023-03-13 12:13:32-04:00,2023-02-11 11:13:16-05:00,2023-02-11 11:13:32-05:00,drums+music,False,"(drums, music)","(rimshot, kit, drum, percussion)","[-1.351285, -0.67553, -0.48624998, -1.5680001,...","[-1.2645626, 0.37058252, -1.060425, 2.4054751,...",0.501911
1,1,3FQ5JJ512Z8CZIGAIVE7BL8KA58NKW,A2JX2VIJWXHS4X,3URJ6VVYV276HHTQ8GM3U9NDDUVO47,2023-03-13 15:22:56-04:00,2023-02-11 14:22:34-05:00,2023-02-11 14:22:56-05:00,Music+Drum+Drum kit,True,"(music, kit, drum)","(rimshot, kit, drum, percussion)","[-0.08255664, 0.5701234, -0.9475999, -0.620833...","[-1.2645626, 0.37058252, -1.060425, 2.4054751,...",0.829466
2,2,3QHK8ZVMI02LIF9V05LZZQL5IGZBLY,A2FEJ4U14OS2M6,33TGB4G0M21DSBM8O8A7LUQY5XHTXD,2023-03-13 12:08:10-04:00,2023-02-11 11:07:33-05:00,2023-02-11 11:08:10-05:00,water running+dog,True,"(water, running)","(water, bathtub, pets, gush)","[-0.108475, 1.6705999, -3.8644, 1.3013095, 2.1...","[-0.26019, 1.8438125, -3.2307024, -1.0729251, ...",0.738576
3,3,36W0OB37HAYFEHTLFYM4TGZ1S2CZHL,A2JX2VIJWXHS4X,33TGB4G0M21DSBM8O8A7LUQY5XHTXD,2023-03-13 15:23:47-04:00,2023-02-11 14:23:36-05:00,2023-02-11 14:23:47-05:00,"Water+Water tap, faucet",True,"(water, tap, faucet)","(water, bathtub, pets, gush)","[0.8942466, 3.9616997, -1.366, 1.9545034, 1.77...","[-0.26019, 1.8438125, -3.2307024, -1.0729251, ...",0.772894
4,4,34V1S5K3G6LKB1RRFH11J18I4L796E,A2FEJ4U14OS2M6,3XAOZ9UYSCBSAU9N0BV5LGBAGB3Q14,2023-03-13 12:09:22-04:00,2023-02-11 11:09:08-05:00,2023-02-11 11:09:22-05:00,cat+speech,True,(speech),"(speech, meow, pets, caterwaul)","[-1.801, -2.1598, -0.33735, -0.33215, 4.8358, ...","[0.21570003, 1.10765, -3.2933624, -4.954488, 1...",0.391362
5,5,3L0KT67Y8S04Z0XWU6WKJNX1VJPSY8,A2JX2VIJWXHS4X,3XAOZ9UYSCBSAU9N0BV5LGBAGB3Q14,2023-03-13 15:24:30-04:00,2023-02-11 14:24:17-05:00,2023-02-11 14:24:30-05:00,Speech+Animal,True,"(speech, animal)","(speech, meow, pets, caterwaul)","[-1.39989, -1.862, -3.431125, -3.278475, 5.097...","[0.21570003, 1.10765, -3.2933624, -4.954488, 1...",0.630436
6,6,3KKG4CDWKWI6WENDWT8MM3O48U194T,A2FEJ4U14OS2M6,3DTJ4WT8CQZ08N4QZV9XJT68ANSZEQ,2023-03-13 12:07:30-04:00,2023-02-11 11:07:15-05:00,2023-02-11 11:07:30-05:00,helicopter,True,(helicopter),"(helicopter, vehicle, aircraft)","[2.2407, 1.9109, -0.51748, -0.70254, -0.82126,...","[1.9864668, 1.8513833, -2.4507601, -0.3573, 1....",0.835747
7,7,3OF2M9AATU86G7UTLJP168FTKJGZKP,A2JX2VIJWXHS4X,3DTJ4WT8CQZ08N4QZV9XJT68ANSZEQ,2023-03-13 15:24:39-04:00,2023-02-11 14:24:30-05:00,2023-02-11 14:24:39-05:00,"Vehicle+Aircraft+Fixed-wing aircraft, airplane",True,"(airplane, fixedwing, vehicle, aircraft)","(helicopter, vehicle, aircraft)","[0.92455745, 1.3760124, -1.9940251, -0.51414, ...","[1.9864668, 1.8513833, -2.4507601, -0.3573, 1....",0.979580
8,8,37FMASSAYQBJSE1PF2OM7ATD1C5BIK,A2FEJ4U14OS2M6,3P0I4CQYWBRIQJCAIMSMP164XNUWO1,2023-03-13 12:07:12-04:00,2023-02-11 11:06:22-05:00,2023-02-11 11:07:12-05:00,man speaking spanish+rooster+knocking wood+out...,True,"(knocking, speaking, rooster, wood, spanish)","(insect, bee, fly)","[-2.151578, -1.9939859, -0.865046, 0.747918, 1...","[2.1473234, 0.10906664, -1.9876295, -0.0710733...",0.352927
9,9,3DIP6YHAPQCOAB1WG7ELVT5K1DDE8F,A2FEJ4U14OS2M6,3XD2A6FGG0EMCFRVWSZ3VM8UWBCS94,2023-03-13 12:12:00-04:00,2023-02-11 11:11:42-05:00,2023-02-11 11:12:00-05:00,old man laughing+coughing,True,"(laughing, coughing)",(cough),"[1.59825, -1.167225, -1.27765, -0.73913, -1.75...","[1.1324, -0.31999, -0.67136, 0.0011585, -0.186...",0.630093


In [45]:
import datetime
df["CompletionTime"] = pd.Series(dtype='object')
time_format = '%y-%m-%d %H:%M:%S'
for index, row in df.iterrows():
    _, _, d1, h1, m1, s1, = [int(x) for x in df.at[index,"AcceptTime"][:-6].replace('-',' ').replace(':',' ').split()]
    _, _, d2, h2, m2, s2 = [int(x) for x in df.at[index,"SubmitTime"][:-6].replace('-',' ').replace(':',' ').split()]
    df.at[index,"CompletionTime"] = datetime.timedelta(days=d2-d1, hours=h2-h1, minutes=m2-m1, seconds=s2-s1).seconds
print(df["CompletionTime"].mean())

24.3


In [46]:
# average score
df.to_csv(out_filename)
df["similarity"].mean()

0.7188624405655367